In [2]:
!export YOLOX_DATADIR=/efs/data/coco_data/
%env YOLOX_DATADIR=/efs/data/coco_data/


env: YOLOX_DATADIR=/efs/data/coco_data/


In [2]:
!echo $YOLOX_DATADIR

/efs/data/coco_data/


In [3]:
model = "yolox-s"
num_accelerators = 1
total_batch_size = 1
num_workers = 1

In [4]:

env_var_options = "NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  " + \
    "NEURON_CC_FLAGS=\'--cache_dir=./compiler_cache --model-type=cnn-training\' " + \
    "XLA_IR_DEBUG=1 XLA_HLO_DEBUG=1"

In [5]:
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1

env: XLA_IR_DEBUG=1
env: XLA_HLO_DEBUG=1


In [1]:
!neuron_parallel_compile

2024-08-29 13:27:01.000593:  311089  INFO ||NEURON_PARALLEL_COMPILE||: Removing existing workdir /tmp/ubuntu/parallel_compile_workdir
usage: neuron_parallel_compile [-h]
                               [--command {collect-and-compile,collect,clear-locks,clear-neffs,compile,clean,scan,scan-with-failed,analyze}]
                               [--num_parallel NUM_PARALLEL]
                               [--node_id NODE_ID] [--world_size WORLD_SIZE]
                               [--static_schedule]
                               [--analyze-output ANALYZE_OUTPUT]
                               [--analyze-verbosity {1,2}]
                               [--parallel_compile_workdir PARALLEL_COMPILE_WORKDIR]
                               [--log_level {INFO,DEBUG,ERROR,WARNING}]

neuron_parallel_compile is an utility to extract graphs from trial run of your script, perform parallel compilation of the graphs, and populate the persistent cache with compiled graphs. Your trial run should be limite

In [7]:
%%time
import subprocess
print("Train model")
RUN_CMD = f"""
   {env_var_options} torchrun --nproc_per_node={num_workers} -m \
   yolox.tools.train \
    -n {model} \
    -d {num_accelerators} \
    -b {total_batch_size} \
    """

print(f'Running command: \n{RUN_CMD}')
if subprocess.check_call(RUN_CMD,shell=True):
   print("There was an error with the fine-tune command")
else:
   print("Fine-tune Successful!!!")

Train model
Running command: 

   NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  NEURON_CC_FLAGS='--cache_dir=./compiler_cache --model-type=cnn-training' XLA_IR_DEBUG=1 XLA_HLO_DEBUG=1 torchrun --nproc_per_node=1 -m    yolox.tools.train     -n yolox-s     -d 1     -b 1     


/bin/sh: 1: nvidia-smi: not found
2024-09-17 18:41:59 | INFO     | yolox.core.trainer:197 - args: Namespace(experiment_name='yolox_s', name='yolox-s', dist_backend='nccl', dist_url=None, batch_size=1, devices=1, exp_file=None, resume=False, ckpt=None, start_epoch=None, num_machines=1, machine_rank=0, fp16=False, cache=None, occupy=False, logger='tensorboard', opts=[])
2024-09-17 18:41:59 | INFO     | yolox.core.trainer:198 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────────────────┼────────────────────────────┤
│ eval_interval     │ 10                         │
├───────────────────┼────────────────────────────┤

------------------------ FINISHED before_iter(), PRINTING METRICS

Metric: IrValueTensorToXlaData
  TotalSamples: 388
  Accumulator: 019ms572.420us
  ValueRate: 568ms975.812us / second
  Rate: 11865.7 / second
  Percentiles: 1%=012.331us; 5%=012.599us; 10%=012.842us; 20%=013.288us; 50%=014.317us; 80%=023.164us; 90%=074.731us; 95%=147.741us; 99%=001ms192.361us
Metric: TensorToData
  TotalSamples: 388
  Accumulator: 018ms240.938us
  ValueRate: 558ms813.545us / second
  Rate: 11865.2 / second
  Percentiles: 1%=011.612us; 5%=011.872us; 10%=012.109us; 20%=012.550us; 50%=013.564us; 80%=022.138us; 90%=073.572us; 95%=146.670us; 99%=001ms191.323us
Metric: UnwrapXlaData
  TotalSamples: 776
  Accumulator: 041.621us
  ValueRate: 001ms272.986us / second
  Rate: 23734.1 / second
  Percentiles: 1%=000.038us; 5%=000.040us; 10%=000.044us; 20%=000.045us; 50%=000.048us; 80%=000.053us; 90%=000.063us; 95%=000.093us; 99%=000.167us
Metric: WrapXlaData
  TotalSamples: 388
  Accumulator: 279.602us
  ValueRate:

2024-09-17 18:57:07 | INFO     | yolox.core.trainer:443 - Save weights to ./YOLOX_outputs/yolox_s


2024-09-17 18:57:08.000318:  1722022  INFO ||NEURON_CACHE||: Compile cache path: ./compiler_cache
2024-09-17 18:57:08.000322:  1722022  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/d19aeedb-b09a-4eb0-85bc-6cb8467b9390/model.MODULE_1011196105946004033+ade7b014.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/d19aeedb-b09a-4eb0-85bc-6cb8467b9390/model.MODULE_1011196105946004033+ade7b014.neff --model-type=cnn-training --verbose=35
......
Compiler status PASS


2024-09-17 18:58:51 | INFO     | yolox.core.trainer:279 - ---> start train epoch 2


------------------------ FINISHED before_iter(), PRINTING METRICS

Metric: DeviceLockWait
  TotalSamples: 104
  Accumulator: 533.646us
  ValueRate: 000.551us / second
  Rate: 0.107296 / second
  Percentiles: 1%=000.566us; 5%=000.767us; 10%=000.890us; 20%=002.780us; 50%=003.553us; 80%=009.902us; 90%=010.238us; 95%=010.594us; 99%=010.958us
Metric: InputOutputAliasCount
  TotalSamples: 2
  Accumulator: 776.00
  ValueRate: 0.89 / second
  Rate: 0.00229838 / second
  Percentiles: 1%=388.00; 5%=388.00; 10%=388.00; 20%=388.00; 50%=388.00; 80%=388.00; 90%=388.00; 95%=388.00; 99%=388.00
Metric: IrValueTensorToXlaData
  TotalSamples: 410
  Accumulator: 019ms495.339us
  ValueRate: 021.485us / second
  Rate: 0.451836 / second
  Percentiles: 1%=012.352us; 5%=012.625us; 10%=012.856us; 20%=013.365us; 50%=014.510us; 80%=029.607us; 90%=072.939us; 95%=146.817us; 99%=770.239us
Metric: TensorToData
  TotalSamples: 430
  Accumulator: 042ms758.871us
  ValueRate: 046.017us / second
  Rate: 0.473846 / second


2024-09-17 19:01:37 | ERROR    | yolox.core.trainer:86 - Exception in training: Bad StatusOr access: INTERNAL: RunNeuronCCImpl: error condition error != 0: <class 'subprocess.CalledProcessError'>: Command '['neuronx-cc', 'compile', '--target=trn1', '--framework=XLA', '/tmp/ubuntu/neuroncc_compile_workdir/9b2b3368-9edd-4b75-8dd1-376c0291d0bc/model.MODULE_18334408780246433435+ade7b014.hlo_module.pb', '--output', '/tmp/ubuntu/neuroncc_compile_workdir/9b2b3368-9edd-4b75-8dd1-376c0291d0bc/model.MODULE_18334408780246433435+ade7b014.neff', '--model-type=cnn-training', '--verbose=35']' returned non-zero exit status 70.
2024-09-17 19:01:37 | INFO     | yolox.core.trainer:261 - Training of experiment is done and the best AP is 0.00
2024-09-17 19:01:37 | ERROR    | yolox.core.launch:98 - An error has been caught in function 'launch', process 'MainProcess' (1722022), thread 'MainThread' (140269788541568):
Traceback (most recent call last):

  File "/usr/lib/python3.10/runpy.py", line 196, in _run_

Fine-tune Successful!!!
CPU times: user 125 ms, sys: 26 ms, total: 150 ms
Wall time: 19min 57s


In [ ]:
%%time
import subprocess
print("Compile model")
COMPILE_CMD = f"""{env_var_options} neuron_parallel_compile --log_level ERROR --num_parallel 1 torchrun --nproc_per_node={num_workers} -m \
   yolox.tools.train \
    -n {model} \
    -d {num_accelerators} \
    -b {total_batch_size} \
    """

print(f'Running command: \n{COMPILE_CMD}')
if subprocess.check_call(COMPILE_CMD,shell=True):
   print("There was an error with the compilation command")
else:
   print("Compilation Success!!!")

In [ ]:
import torch_xla.distributed.xla_backend
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch

In [ ]:
gt_classes = torch.tensor([41., 50.], device='xla:0')
gt_classes

In [ ]:
gt_classes.norm()

In [ ]:
gt_classes.max()

In [ ]:
import torch.nn.functional as F
gt_classes = torch.tensor([ 0.,  0., 14., 56., 60.])
num_classes = 80

In [ ]:
gt_cls_per_image = (
    F.one_hot((gt_classes).to(torch.int64), num_classes)
    .float()
)